# Convolutional Neural Network

## Dataset

### Layout

* Images:
	* Dog
    * Cat
* 1000s of images
	* Training set
	    * 4000 images each for dogs and cats
    * Test set
        * 1000 images each for dogs and cat

### Goals

* Build a CNN model to classify images for a dog or cat

---

## Import Libraries

In [25]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [26]:
tf.__version__

'2.20.0'

---

## Data Preprocessing

### Preprocessing Training Set

#### Image Augmentation

* Transformations will be applied to images only in the training set
    * This is to avoid over fitting
    * Otherwise, there will be a huge difference in accuracy of the training and test sets:
        * Close to $98\%$ accuracy on the training set
        * Much lower accuracy on the test set
* Geometric transformations are applied to the training set images:
    * For example, zoom, rotations, etc. on the images
    * First, transvections to shift pixels
    * Next, rotations with horizontal flips and zoom-in-and-out
* These transformations are called **image augmentation**
* The goal is to augment the diversity of the training set images

#### Image Generation

* The `ImageDataGenerator` class from the `preprocessing.image` module of the Keras library is used to generate images and perform image augmentation
    * Parameters
        * `rescale` applies feature scaling to each pixel
            * Each pixel has a value between $0$ and $255$
            * Each pixel value is divided by $255$
            * This will normalize the pixel values in images
        * `shear_range` randomly applies a range of shear transformations that shift pixels by a specified position to images
        * `zoom_range` randomly applies a range of zooming to images
        * `horizontal_flip` indicates a value to flip images horizontally
* The `train_datagen` variable defines an instance of the `ImageDataGenerator` class
* The training dataset will be imported from the dataset directory containing the directory of training set images
    * The `flow_from_directory` method on the `ImageDataGenerator` class recursively loads images from a specified directory
        * Parameters
            * `directory` is the root directory of images to process
            * `target_size` performs image resizing by specifying the target pixel size of the images
                * This makes the image processing less computationally intensive
            * `batch_size` indicates the number of images to process per batch
            * `class_mode` specifies the classification mode: `binary` or `categorical`
* The `training_set` variable is the training set of the images

In [27]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

In [28]:
training_set = train_datagen.flow_from_directory(
    directory='dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
)

Found 8000 images belonging to 2 classes.


### Preprocessing Test Set

* Image augmentation is not applied to test set images so those parameters are omitted

In [29]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_set = test_datagen.flow_from_directory(
    directory='dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
)

Found 2000 images belonging to 2 classes.


---

## Build CNN

### Initialize CNN

* The `Sequential` class is from the `models` module of the Keras library and allows one to construct a neural network made of a sequence of layers
* The `cnn` variable is an object instance of the `Sequential` class

In [30]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

* The `Conv2D` class is from the `layers` module of the Keras library and creates a convolutional layer
    * Parameters
        * `filters` is the number of feature detectors used to perform convolution
        * `kernel_size` is the size of the array (matrices as row and columns) for a feature detector
        * `strides` is the number of pixels to move a feature detector across and up/down an image. Default value of $1$ will be used.
        * `activation` is the activation function to apply after convolution
        * `input_shape` is the dimensions of an input image in the RGB 3 dimensions of color:
            * `width` = $64$ pixels
            * `height` = $64$ pixels
            * `colors` = $3$ indicates colored images. $0$ = black and $1$ = white.

In [31]:
cnn.add(
    tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        activation='relu',
        input_shape=(64, 64, 3)
    )
)

### Step 2 - Pooling

* The `MaxPoolConv2D` class is from the `layers` module of the Keras library and creates a pooling layer using max pooling
    * Parameters
        * `pool_size` is the size of the array (matrices as row and columns) of the pool window (frame)
        * `strides` is the number of pixels to move the pool window across and up/down an image

In [32]:
cnn.add(
    tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=(2, 2)
    )
)

### Add 2nd Convolutional Layer

* The `input_shape` parameter is omitted because is the shape is only defined on the first convolutional layer in the CNN

In [33]:
cnn.add(
    tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        activation='relu',
    )
)

In [34]:
cnn.add(
    tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=(2, 2)
    )
)

### Step 3 - Flattening

* The `Flatten` class is from the `layers` module of the Keras library and creates a flattening layer
    * No parameters are required for an instance of this class

In [35]:
cnn.add(
    tf.keras.layers.Flatten()
)

### Step 4 - Full Connection

* The `Dense` class is from the `layers` module of the Keras library and allows one to add a fully connected layer to an ANN
    * An object instance of this class is used to construct a connected layer
    * Parameters
        * `units` defines the number of hidden neurons in a hidden layer
            * $128$ neurons are used to achieve a higher level of accuracy
        * `activation` defines the activation function
            * Rectifier activation function (`relu`) will be used for hidden layers

In [36]:
cnn.add(
    tf.keras.layers.Dense(
        units=128,
        activation='relu'
    )
)

### Step 5 - Output Layer

* The same `Dense` class is used to construct the output layer except it:
    * Has $1$ neuron
        * Since there is only $1$ dependent variable
    * Has Sigmoid activation function
        * Used when making predictions that are binary for classification

In [37]:
cnn.add(
    tf.keras.layers.Dense(
        units=1,
        activation='sigmoid'
    )
)

---

## Training CNN

### Compile CNN

### Train CNN on Training Set and Evaluate on Test Set

---

## Make Predictions

### Make Single Prediction